# District Cleaning — Andhra Pradesh

This notebook standardizes **district names** for this state across:
- Enrolment data
- Demographic update data
- Biometric update data

**All data is saved back to the same cleaned files.**

In [14]:
import pandas as pd
from pathlib import Path

pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)

CLEAN_DIR = Path("../../data/processed/cleaned")

enrol_df = pd.read_csv(CLEAN_DIR / "enrolment_clean.csv")
demo_df  = pd.read_csv(CLEAN_DIR / "demographic_clean.csv")
bio_df   = pd.read_csv(CLEAN_DIR / "biometric_clean.csv")

for df in [enrol_df, demo_df, bio_df]:
    df["state"] = df["state"].astype(str).str.strip().str.title()
    df["district"] = df["district"].astype(str).str.strip().str.title()

print("✅ All datasets loaded and normalized (Title Case)")


✅ All datasets loaded and normalized (Title Case)


In [18]:
STATE_NAME = "Andhra Pradesh"

districts = sorted(
    set(
        enrol_df.loc[enrol_df["state"] == STATE_NAME, "district"].dropna()
        .tolist()
    )
)

print(f"State: {STATE_NAME}")
print(f"Number of unique districts: {len(districts)}")

pd.DataFrame(
    {"District Name": districts}
)


State: Andhra Pradesh
Number of unique districts: 26


,District Name
0,Alluri Sitharama Raju
1,Anakapalli
2,Ananthapuramu
3,Annamayya
4,Bapatla
5,Chittoor
6,Dr. B. R. Ambedkar Konaseema
7,East Godavari
8,Eluru
9,Guntur


## District Mapping

Add mappings in **Title Case only**.

Format:
```python
DISTRICT_MAPPING = {
    "Correct District": ["Wrong Name 1", "Wrong Name 2"],
}
```

In [16]:
DISTRICT_MAPPING = {
    # "Correct District": ["Wrong Variant 1", "Wrong Variant 2"]
    "Ananthapuramu": ["Anantapur", "Ananthapur"],
    "Sri Potti Sriramulu Nellore": ["Nellore", "Spsr Nellore"],
    "NTR": ["Ntr"],
    "Visakhapatnam": ["Visakhapatanam"],
    "Y.S.R. Kadapa": ["Cuddapah", "Y. S. R"]
}

def apply_mapping(df, state, mapping, label):
    total = 0
    for correct, wrongs in mapping.items():
        mask = (
            (df["state"] == state) &
            (df["district"].isin(wrongs))
        )
        count = mask.sum()
        df.loc[mask, "district"] = correct
        total += count
        if count > 0:
            print(f"✔ {label} → {correct} : {count} rows fixed")
    return total

total_fixes = 0
total_fixes += apply_mapping(enrol_df, STATE_NAME, DISTRICT_MAPPING, "Enrolment")
total_fixes += apply_mapping(demo_df,  STATE_NAME, DISTRICT_MAPPING, "Demographic")
total_fixes += apply_mapping(bio_df,   STATE_NAME, DISTRICT_MAPPING, "Biometric")

print(f"✅ Total fixes in {STATE_NAME}: {total_fixes}")


✔ Enrolment → Ananthapuramu : 3877 rows fixed
✔ Enrolment → Sri Potti Sriramulu Nellore : 2676 rows fixed
✔ Enrolment → Visakhapatnam : 1 rows fixed
✔ Enrolment → Y.S.R. Kadapa : 3315 rows fixed
✔ Demographic → Ananthapuramu : 10093 rows fixed
✔ Demographic → Sri Potti Sriramulu Nellore : 5114 rows fixed
✔ Demographic → Y.S.R. Kadapa : 7472 rows fixed
✅ Total fixes in Andhra Pradesh: 32548


In [17]:
enrol_df.to_csv(CLEAN_DIR / "enrolment_clean.csv", index=False)
demo_df.to_csv(CLEAN_DIR / "demographic_clean.csv", index=False)
bio_df.to_csv(CLEAN_DIR / "biometric_clean.csv", index=False)

print("💾 All cleaned files saved successfully (overwritten)")


💾 All cleaned files saved successfully (overwritten)
